# Modelo de Machine Learning para predecir la deserción de clientes de Beta Bank

# Contenido 

* [Introducción](#)
* [Objetivos](#)
* [Etapas](#)
* [Inicialización: descripción de los datos](#)
* [Preprocesamiento de datos](#)
* [Análisis exploratorio de datos](#)
* [Estandarización y segmentación de datos](#)
* [Desequilibrio de clases y modelos sin equilibrar](#)
* [Modelamiento con equilibrio de clases](#)
* [Prueba final del modelo](#)
* [Extra: prueba de cordura](#)
* [Conclusiones](#)

# Introducción

El banco Beta Bank, uno de los más importantes de su país, nos cuenta que últimamente lidia con problemas para retener sus clientes y que estos han estado dejando el banco poco a poco, cada mes. Realizaron un estudio en el que descubrieron que es más económico tratar de salvar a los clientes actuales en lugar de atraer nuevos, por el cual decidieron priorizar este objetivo, el cual implica que necesitan una forma que les ayude a predecir si un cliente dejará el banco pronto. Para esto, desarrollaremos un modelo de Machine Learning (de clasificación) que nos ayude con esta tarea, en base a los datos del comportamiento pasado de los clientes del banco.

# Objetivos

Nuestro principal objetivo será determinar el modelo con el máximo valor F1 posible, teniendo como umbral mínimo un valor F1 de 0.59.

También tendremos en cuenta el desequilibrio de clases en este proyecto, y analizaremos el modelamiento teniendo en cuenta o no este problema.

A su vez, compararemos el valor F1 obtenido y lo compararemos con la métrica AUC-ROC para medir el desempeño del modelo. 

# Inicialización: descripción de los datos

**Carga de librerías necesarias**

In [9]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt  # librerías estándar para manipular datos

from sklearn.preprocessing import StandardScaler  # estandarizador de sklearn
from sklearn.utils import shuffle  # mezclador de sklearn 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import f1_score, roc_curve, roc_auc_score, precision_score, recall_score  # métricas que usaremos 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression  # modelos necesarios

import warnings
warnings.filterwarnings('ignore')  # ignoraremos algunos warnings

## Cargar los datos

Beta Bank nos proporcionó el siguiente dataset:

In [3]:
try:
    data_beta = pd.read_csv("Churn.csv")
except:
    data_beta = pd.read_csv("/datasets/Churn.csv")

## Exploración inicial de datos

Según la documentación, en el dataset tenemos la siguiente información:

* ``RowNumber``: índice de cadena de datos.
* ``CustomerId``: identificador de cliente único.
* ``Surname``: apellido.
* ``CreditScore``: valor de crédito.
* ``Geography``: país de residencia.
* ``Gender``: género.
* ``Age``: edad.
* ``Tenure``: período durante el cual ha madurado el depósito a plazo fijo de un cliente (años).
* ``Balance``: saldo de cuenta.
* ``NumOfProducts``: número de productos bancarios utilizados por el clientes.
* ``HasCrCard``: si el cliente tiene una tarjeta de crédito (1 = sí, 0 = no).
* ``IsActiveMember``: actividad del cliente (1 = sí, 0 = no).
* ``EstimatedSalary``: salario estimado.
* ``Exited``: si el cliente se ha ido (1 = sí, 0 = no).

Ahora exploramos los datos:

In [4]:
data_beta.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [5]:
data_beta.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


In [6]:
data_beta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


De las columnas, no notamos problemas evidentes: la mayor parte de ellas tienen valores razonables y sus distribuciones no parecen mostrar sesgos o asimetrías. 

El problema más grande resulta en la columna ``Tenure``, que muestra tener valores ausentes. Pensaremos en cómo resolver esto pronto.

Otro detalle no menor es que el nombre de las columnas tiene un formato incorrecto al usar mayúsculas, cosa que también solucionaremos.

Por último, parecen existir columnas con información que no será útil al momento de realizar el modelo: ``RowNumber``, ``CustomerId`` y ``Surname`` no deberían tener una relación significativa *a priori* para ayudarnos a predecir nuestro target ``Exited``. 

A continuación, daremos un vistazo al problema de los valores duplicados, y también de los ausentes de la columna ``Tenure``:

In [10]:
data_beta.duplicated().sum()

0

Ya que no tenemos duplicados, veremos la distribución de ``Tenure`` y sus valores ausentes:

In [11]:
data_beta['Tenure'].value_counts(dropna=False, normalize=True)

1.0     0.0952
2.0     0.0950
8.0     0.0933
3.0     0.0928
5.0     0.0927
7.0     0.0925
NaN     0.0909
4.0     0.0885
9.0     0.0882
6.0     0.0881
10.0    0.0446
0.0     0.0382
Name: Tenure, dtype: float64

Relativizando, parece que tenemos tantos valores ausentes como los valores más frecuentes de la tabla (alrededor del 9 %). De esta distribución de la columna, resulta interesante que los clientes que tienen un depósito a plazo fijo por 8 años estén entre los más frecuentes, cuando se esperaría una relación inversa con el tiempo de forma más clara. Por ahora, continuaremos con el preprocesamiento.

## Conclusiones del apartado

En este pequeño apartado, encontramos que la columna ``Tenure`` cuenta con un 9 % de sus valores como ausentes, siendo este el principal problema hallado. También las columnas presentan un formato incorrecto en sus nombres y existen columnas que en teoría no están relacionadas con nuestra variable objetivo. 

Fuera de esto, los datos parecen estar correctos y no muestran más problemas. El tratamiento de estos problemas viene a continuación.

# Preprocesamiento de datos

Tenemos varios problemas a resolver en este apartado, pero empezaremos por el que nos facilitará el proceso primero: eliminar columnas.

## Columnas prescindibles

Habíamos expuesto que las columnas ``RowNumber``, ``CustomerId`` y ``Surname`` no deberían tener un impacto significativo en la determinación de ``Exited``. En el sentido teórico y a sentido común, este tipo de variables no deberían tener relación con el hecho de si un cliente va a dejar el banco o no, y si existiera una relación de cualquier tipo, esta podría considerarse no causal o espuria. Por ejemplo, el apellido de un cliente no determina si este dejará el banco. 

Debido a todo lo explicado, lo correcto sería directamente prescindir de estas columnas en este paso, a manera primer *feature engineering* en el proceso de modelamiento.

In [13]:
data_beta = data_beta.drop(columns=['RowNumber', 'CustomerId', 'Surname'])

In [14]:
data_beta.columns

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Exited'],
      dtype='object')

## Corrigiendo los nombres de las columnas

No solo volveremos minúsculas los nombres, sino que usaremos los estándares que los creadores de Python recomiendan para los nombres de las columnas:

In [15]:
data_beta = data_beta.rename(
    columns= {
        'CreditScore' : 'credit_score',
        'NumOfProducts' : 'num_of_products',
        'HasCrCard' : 'has_cr_card',
        'IsActiveMember': 'is_active_member',
        'EstimatedSalary': 'estimated_salary'
    }
)

In [16]:
data_beta.columns = data_beta.columns.str.lower()

In [17]:
data_beta.columns

Index(['credit_score', 'geography', 'gender', 'age', 'tenure', 'balance',
       'num_of_products', 'has_cr_card', 'is_active_member',
       'estimated_salary', 'exited'],
      dtype='object')

Los problemas de las columnas han sido corregidos. El paso siguiente es tratar los valores ausentes.

## Valores ausentes de ``tenure``

Antes de pensar en el método a usar para rellenar sus valores ausentes, necesitamos probar si estos tienen una causa detrás que los relaciona a otra columna, es decir, si estos son causados o afectados por alguna otra columna. Para esto, usaremos un bucle y observaremos si existen cambios con y sin valores ausentes:

In [19]:
for column in data_beta:
    print(data_beta[column].value_counts(dropna=False, normalize=True))
    print()

850    0.0233
678    0.0063
655    0.0054
705    0.0053
667    0.0053
        ...  
404    0.0001
351    0.0001
365    0.0001
417    0.0001
419    0.0001
Name: credit_score, Length: 460, dtype: float64

France     0.5014
Germany    0.2509
Spain      0.2477
Name: geography, dtype: float64

Male      0.5457
Female    0.4543
Name: gender, dtype: float64

37    0.0478
38    0.0477
35    0.0474
36    0.0456
34    0.0447
       ...  
92    0.0002
82    0.0001
88    0.0001
85    0.0001
83    0.0001
Name: age, Length: 70, dtype: float64

1.0     0.0952
2.0     0.0950
8.0     0.0933
3.0     0.0928
5.0     0.0927
7.0     0.0925
NaN     0.0909
4.0     0.0885
9.0     0.0882
6.0     0.0881
10.0    0.0446
0.0     0.0382
Name: tenure, dtype: float64

0.00         0.3617
130170.82    0.0002
105473.74    0.0002
85304.27     0.0001
159397.75    0.0001
              ...  
81556.89     0.0001
112687.69    0.0001
108698.96    0.0001
238387.56    0.0001
130142.79    0.0001
Name: balance, Length: 6382, dtype

In [20]:
for column in data_beta:
    print(data_beta.dropna(subset='tenure')[column].value_counts(dropna=False, normalize=True))
    print()

850    0.02310
678    0.00660
655    0.00561
705    0.00528
683    0.00517
        ...   
407    0.00011
417    0.00011
365    0.00011
440    0.00011
419    0.00011
Name: credit_score, Length: 458, dtype: float64

France     0.500495
Germany    0.252227
Spain      0.247278
Name: geography, dtype: float64

Male      0.547135
Female    0.452865
Name: gender, dtype: float64

37    0.04785
35    0.04741
38    0.04664
34    0.04532
33    0.04433
       ...   
82    0.00011
88    0.00011
85    0.00011
92    0.00011
83    0.00011
Name: age, Length: 70, dtype: float64

1.0     0.104719
2.0     0.104499
8.0     0.102629
3.0     0.102079
5.0     0.101969
7.0     0.101749
4.0     0.097349
9.0     0.097019
6.0     0.096909
10.0    0.049060
0.0     0.042020
Name: tenure, dtype: float64

0.00         0.361126
105473.74    0.000220
130170.82    0.000220
108935.39    0.000110
114453.58    0.000110
               ...   
176024.05    0.000110
107499.70    0.000110
98807.45     0.000110
136596.85    0.00